In [1]:
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
import numpy as np
from matplotlib import pyplot as plt
import random

In [2]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255

y_orig_train = np.copy(y_train)
y_orig_test = np.copy(y_test)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

model = load_model("../saved_models/LeNet.h5")

In [3]:
print(x_test.shape)

(10000, 28, 28, 1)


In [4]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02577946839928336
Test accuracy: 0.992


In [5]:
def mutation_manager(data, indices):
    cols = data.shape[1]
    for x in indices:
        row_no = int(x / cols)
        col_no = x % cols
        data[row_no, col_no] = random.uniform(0, 1.0)
    return data

In [6]:
def roulette_select(population, fitnesses, num):
    total_fitness = float(sum(fitnesses))
    rel_fitness = [f/total_fitness for f in fitnesses]
    # Generate probability intervals for each individual
    probs = [sum(rel_fitness[:i+1]) for i in range(len(rel_fitness))]
    # Draw new population
    new_population = []
    for n in range(num):
        r = np.random.random()
        for (i, individual) in enumerate(population):
            if r <= probs[i]:
                new_population.append((individual, fitnesses[i]))
                break
    return new_population

In [7]:
def crossover_manager(genome1, genome2):
    random_index_to_cut = int(np.random.random() * len(genome1))
    new1 = list(genome1[:random_index_to_cut]) + list(genome2[random_index_to_cut:])
    new2 = list(genome2[:random_index_to_cut]) + list(genome1[random_index_to_cut:])
    
    return np.asarray(new1), np.asarray(new2)

In [8]:
def genetic_op(data, required_val, mutation_rate, crossover_rate):
    # Crossover operation
    no_of_crossovers = int(data.shape[0] * crossover_rate)
    indices_to_crossover = np.random.choice(data.shape[0], no_of_crossovers, replace=False)
    data_to_be_crossover = data[indices_to_crossover]

    for x in range(0, len(indices_to_crossover) - 1, 2):
        genome1 = data_to_be_crossover[x]
        genome2 = data_to_be_crossover[x+1]
        ind1 = indices_to_crossover[x]
        ind2 = indices_to_crossover[x+1]
        new1, new2 = crossover_manager(genome1, genome2)
        data[ind1] = new1
        data[ind2] = new2
    
    # Now for the mutation operation
    no_of_mutations = int(data.shape[0] * data.shape[1] * mutation_rate)
    indices_to_mutate = np.random.choice(data.shape[0] * data.shape[1], no_of_mutations, replace=False)
    data = mutation_manager(data, indices_to_mutate)
    
    # Finally fitness
    y = data.reshape(data.shape[0], 28, 28, 1)
    r = model.predict(y)
    fitness = r[:, required_val]

    return data, fitness    

In [9]:
def add_noise(img, _):
    random_noise = np.minimum(np.maximum(np.random.normal(loc=0, scale=2.5,size=img.shape),0),1)    
    return random_noise

In [11]:
# First we need to set some parameters for GA
def generate_noisy_images(new_num):

    init_crossover_rate = 0.015
    init_mutation_rate = 0.05
    population_size = 50
    generations = 200
    promotion_rate = 0.1
    multiplier = int(1 / promotion_rate)

    fit_num = []
    # Now for each number we have to create noisy initial population.
    try:
        for numb in range(10):
            mutation_rate = init_mutation_rate
            crossover_rate = init_crossover_rate
            required_val = numb
#             print("Now evaluating for number", required_val)
            initial_popl = new_num[numb].reshape(
                new_num[numb].shape[0],
                new_num[numb].shape[1] * new_num[numb].shape[2]
            )
            initial_popl = np.tile(initial_popl, (population_size // new_num[numb].shape[0], 1))
            prev_popl = [[0, 0]]

            for gen in range(generations):
                change = False
                while not change:
                    data, fitness = genetic_op(initial_popl, required_val, crossover_rate, mutation_rate)
                    data_with_fitness = zip(data, fitness)
                    data_with_fitness = sorted(data_with_fitness, key=lambda x: x[1], reverse=True)

                    # Now select the promoted data for the next round
                    promotion = data_with_fitness[:int(population_size * promotion_rate)]
                    # promotion = roulette_select(list(data), fitness, int(population_size * promotion_rate))

                    if prev_popl[0][1] > promotion[0][1]:
                        initial_popl = np.asarray([x[0] for x in prev_popl])
                    else:
                        only_data = [x[0] for x in promotion]
                        initial_popl = np.asarray(only_data * multiplier)
                        prev_popl = promotion * multiplier
                        change = True

    #             print("Max fitness for gen {} is {}".format(gen, promotion[0][1]))
                if (promotion[0][1] > .1 and gen > 150) or (promotion[0][1] > .9999):
#                     print("Adding the image to the new array...")
                    fit_num.append(promotion[0][0])
                    break
        return fit_num
    except KeyboardInterrupt:
        print("GA stopped!")

In [12]:
"""
Now to generate the fooling images, we can start from the scratch.
But doing so will take a lot of time. So we decided to start with
a population of noisy dataset
"""
# First we will split the data
num = []

for x in range(10):
    indices = np.argwhere(y_orig_train == x).reshape((-1,))
    num.append(x_train[0:1000])

num = np.asarray(num)

In [13]:
for j in range(1,10):
    all_num = []
    for i in range(j*100,(j+1)*100):
        new_num = []
        for x in range(10):
            numbers = num[x][i:i+1]
            final_image = np.apply_over_axes(add_noise, numbers, [0])
    #         plt.imshow(final_image.squeeze(),'gray')
    #         plt.show()
            new_num.append(final_image)
        new_num = np.asarray(new_num)
        fit_num = generate_noisy_images(new_num)
        if i % 50 == 0:
            print('第{}次'.format(i))
        all_num.append(fit_num)

    aa = np.array(all_num)
    irregular = aa.reshape(aa.shape[0]*aa.shape[1],28,28,1)
    np.save("mnist_fooling_images%s.npy"%(j), irregular)

第100次
第150次
第200次
第250次
第300次
第350次
第400次
第450次
第500次
第550次
第600次
第650次
第700次
第750次
第800次
第850次
第900次
第950次
